# レース情報のスクレイピング(日付、距離、天候等)

## スクレイピング対象の調査

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
race_id = '201901010101'
url = 'https://db.netkeiba.com/race/' + race_id
html = requests.get(url)
html.encoding = 'EUC-JP'
soup = BeautifulSoup(html.text, 'html.parser')

In [3]:
soup

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html id="html" lang="ja" xml:lang="ja" xmlns="http://www.w3.org/1999/xhtml">
<head>
<title>2歳未勝利｜2019年7月27日 | 競馬データベース - netkeiba.com</title>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="ja" http-equiv="content-language">
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="text/javascript" http-equiv="content-script-type"/>
<meta content="text/css" http-equiv="content-style-type"/>
<meta content="競馬データベースです。競走馬、騎手、レース、調教師、馬主。" name="description"/>
<meta content="競馬情報,競走馬,騎手,レース,調教師,検索,データベース,JRA,netkeiba.com" name="keywords"/>
<link href="https://cdn.netkeiba.com/img.db/common/css/reset.css?20160421" media="all" rel="stylesheet" type="text/css"/>
<link href="https://cdn.netkeiba.com/img.db/common/css/common.css?2020" media="all" rel="stylesheet" type="text/css"/>
<link href="https://cdn.netkeiba.com

In [ ]:
# divタグの、クラス名data_introの、pタグの一覧データリストを取得し、表示
soup.find('div', attrs={'class':'data_intro'}).find_all('p')

In [7]:
# divタグの、クラス名data_introの、pタグの一覧データリストを取得し、その中の先頭テキストを表示
soup.find('div', attrs={'class':'data_intro'}).find_all('p')[0].text

'\n\n芝右1800m\xa0/\xa0天候 : 曇\xa0/\xa0芝 : 良\xa0/\xa0発走 : 09:50\n\n\n'

In [8]:
text = soup.find('div', attrs={'class':'data_intro'}).find_all('p')[0].text

In [11]:
import re
# 英数字でマッチしなくなるまで  \wが任意の英数字
re.findall(r'\w+', text)

['芝右1800m', '天候', '曇', '芝', '良', '発走', '09', '50']

In [12]:
text2 = soup.find('div', attrs={'class':'data_intro'}).find_all('p')[1].text

In [13]:
re.findall(r'\w+', text2)

['2019年7月27日', '1回札幌1日目', '2歳未勝利', '混', '指', '馬齢']

# スクレイピング本番

In [3]:
import pandas as pd
results  = pd.read_pickle('2019_result_raw.pickle')

In [34]:
race_id_list = results.index.unique()

In [35]:
race_id_list

Index(['201901010101', '201901010102', '201901010103', '201901010104',
       '201901010105', '201901010106', '201901010107', '201901010108',
       '201901010109', '201901010110',
       ...
       '201910021203', '201910021204', '201910021205', '201910021206',
       '201910021207', '201910021208', '201910021209', '201910021210',
       '201910021211', '201910021212'],
      dtype='object', length=3452)

In [36]:
# 必要ライブラリインポート
import time
import re
from tqdm import tqdm

In [37]:
# 情報取得後、整形したデータを返す
def scrape_race_info(race_id_list):

    race_infos = {}
    for race_id in tqdm(race_id_list):
        try:
            url = 'https://db.netkeiba.com/race/' + race_id
            html = requests.get(url)
            html.encoding = 'EUC-JP'
            soup = BeautifulSoup(html.text, 'html.parser')

            texts = soup.find('div', attrs={'class':'data_intro'}).find_all('p')[0].text + soup.find('div', attrs={'class':'data_intro'}).find_all('p')[1].text
            info = re.findall(r'\w+', texts)
            race_infos[race_id] = info

            info_dict = {}
            for text in info:
                if text in ['芝', 'ダート']:
                    info_dict['race_type'] = text
                if '障' in text:
                    info_dict['race_type'] = '障害'
                if 'm' in text:
                    info_dict['course_len'] = re.findall(r'\d+', text)[0]
                if text in ['良','稍重','重','不良']:
                    info_dict['ground_state'] = text
                if text in ['曇','晴','雨','小雨','小雪','雪']:
                    info_dict['weather'] = text
                if '年' in text:
                    info_dict['date'] = text
                race_infos[race_id] = info_dict
                time.sleep(1)
                
        except IndexError:
            continue
        except Exception as e:
            print(e)
            break
        except:
                break
        
    return race_infos

In [38]:
race_infos = scrape_race_info(race_id_list)

100%|████████████████████████████████████| 3452/3452 [14:05:25<00:00, 14.69s/it]


In [39]:
race_infos

{'201901010101': {'course_len': '1800',
  'weather': '曇',
  'race_type': '芝',
  'ground_state': '良',
  'date': '2019年7月27日'},
 '201901010102': {'course_len': '1000',
  'weather': '曇',
  'race_type': 'ダート',
  'ground_state': '良',
  'date': '2019年7月27日'},
 '201901010103': {'course_len': '2400',
  'weather': '曇',
  'race_type': 'ダート',
  'ground_state': '良',
  'date': '2019年7月27日'},
 '201901010104': {'course_len': '2000',
  'weather': '晴',
  'race_type': '芝',
  'ground_state': '良',
  'date': '2019年7月27日'},
 '201901010105': {'course_len': '1500',
  'weather': '晴',
  'race_type': '芝',
  'ground_state': '良',
  'date': '2019年7月27日'},
 '201901010106': {'course_len': '1700',
  'weather': '晴',
  'race_type': 'ダート',
  'ground_state': '良',
  'date': '2019年7月27日'},
 '201901010107': {'course_len': '1000',
  'weather': '晴',
  'race_type': 'ダート',
  'ground_state': '良',
  'date': '2019年7月27日'},
 '201901010108': {'course_len': '1200',
  'weather': '晴',
  'race_type': '芝',
  'ground_state': '良',
  'date':

In [47]:
df = pd.DataFrame(race_infos).T

In [49]:
df

In [50]:
results_addinfo = results.merge(df, left_index=True, right_index=True, how='inner')

In [51]:
results_addinfo

,着順,枠番,馬番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人気,馬体重,調教師,course_len,weather,race_type,ground_state,date
201901010101,1,1,1,ゴルコンダ,牡2,54.0,ルメール,1:48.3,NaN,1.4,1.0,518(-16),[東] 木村哲也,1800,曇,芝,良,2019年7月27日
201901010101,2,3,3,プントファイヤー,牡2,54.0,岩田康誠,1:50.1,大,3.5,2.0,496(-8),[東] 手塚貴久,1800,曇,芝,良,2019年7月27日
201901010101,3,4,4,ラグリマスネグラス,牡2,51.0,団野大成,1:50.9,5,46.6,6.0,546(+6),[東] 藤沢和雄,1800,曇,芝,良,2019年7月27日
201901010101,4,8,9,キタノコドウ,牡2,51.0,菅原明良,1:51.5,3.1/2,56.8,7.0,458(-8),[東] 高木登,1800,曇,芝,良,2019年7月27日
201901010101,5,5,5,ネモフィラブルー,牡2,54.0,川島信二,1:51.7,1.1/2,140.3,9.0,436(0),[西] 矢作芳人,1800,曇,芝,良,2019年7月27日
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201910021212,12,6,11,スリープレッピー,セ6,56.0,森裕太朗,2:45.7,1/2,120.3,15.0,458(+8),[西] 藤沢則雄,2600,曇,芝,重,2019年9月1日
201910021212,13,1,1,バリオラージュ,牡5,54.0,斎藤新,2:46.0,1.3/4,7.5,4.0,460(+2),[西] 角居勝彦,2600,曇,芝,重,2019年9月1日
201910021212,14,2,3,サンライズアミーゴ,牡4,54.0,亀田温心,2:46.2,1,99.2,12.0,478(+14),[西] 牧浦充徳,2600,曇,芝,重,2019年9月1日
201910021212,15,6,12,トロハ,牝3,52.0,武豊,2:46.2,クビ,17.5,8.0,468(+2),[西] 浜田多実,2600,曇,芝,重,2019年9月1日


In [52]:
results_addinfo.isnull().sum()

着順                 0
枠番                 0
馬番                 0
馬名                 0
性齢                 0
斤量                 0
騎手                 0
タイム              452
着差              3906
単勝                 0
人気               229
馬体重                0
調教師                0
course_len         0
weather            0
race_type          0
ground_state       0
date               0
dtype: int64

In [54]:
results_addinfo.to_pickle('2019_result_combined_raw.pickle')